# [`wordplay` 🎮 💬](https://github.com/saforem2/wordplay): Shakespeare

✍️

[Sam Foreman](https://samforeman.me)
(\[[ALCF](https://alcf.anl.gov/about/people/sam-foreman)\](<https://alcf.anl.gov/about/people/sam-foreman>))  
2025-07-22

We will be using the [Shakespeare
dataset](https://github.com/saforem2/wordplay/blob/main/data/shakespeare/readme.md)
to train a (~ small) 10M param LLM *from scratch*.

<img src="https://github.com/saforem2/wordplay/blob/main/assets/shakespeare.jpeg?raw=true" width="45%" align="center" /><br>

Image generated from
[stabilityai/stable-diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
on [🤗 Spaces](https://huggingface.co/spaces).<br>

<details closed>

<summary>

Prompt Details

</summary>

<ul>

<li>

Prompt:

</li>

<t><q> Shakespeare himself, dressed in full Shakespearean garb, writing
code at a modern workstation with multiple monitors, hacking away
profusely, backlit, high quality for publication </q></t>

<li>

Negative Prompt:

</li>

<t><q> low quality, 3d, photorealistic, ugly </q></t>

</ul>

</details>

## Install / Setup

<b>Warning!</b><br>

**IF YOU ARE EXECUTING ON GOOGLE COLAB**:

You will need to restart your runtime (`Runtime` $\rightarrow\,$
`Restart runtime`)  
*after* executing the following cell:

In [1]:
%%bash

python3 -c 'import wordplay; print(wordplay.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has wordplay installed. Nothing to do."
else
    echo "Does not have wordplay installed. Installing..."
    git clone 'https://github.com/saforem2/wordplay'
    python3 wordplay/data/shakespeare_char/prepare.py
    python3 wordplay/data/shakespeare/prepare.py
    python3 -m pip install deepspeed
    python3 -m pip install -e wordplay
fi

/Users/samforeman/projects/saforem2/wordplay/src/wordplay/__init__.py
Has wordplay installed. Nothing to do.

## Post Install

If installed correctly, you should be able to:

``` python
>>> import wordplay
>>> wordplay.__file__
'/path/to/wordplay/src/wordplay/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import ezpz

os.environ['COLORTERM'] = 'truecolor'
if sys.platform == 'darwin':
    # If running on MacOS:
    # os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
    os.environ['TORCH_DEVICE'] = 'cpu'
# -----------------------------------------------

logger = ezpz.get_logger()

import wordplay
logger.info(wordplay.__file__)

[ 2025-07-22 22:23:53,364061 ][ I ][ ezpz / __init__ : 265 : ezpz ] Setting logging level to 'INFO' on 'RANK == 0'

[ 2025-07-22 22:23:53,390701 ][ I ][ ezpz / __init__ : 266 : ezpz ] Setting logging level to 'CRITICAL' on all others 'RANK != 0'

[ 2025-07-22 22:23:53,395170 ][ I ][ ipykernel_52985 / 2338663768 : 17 : ezpz.log ] /Users/samforeman/projects/saforem2/wordplay/src/wordplay/ __init__.py

## Build Trainer

Explicitly, we:

1.  `setup_torch(...)`
2.  Build `cfg: DictConfig = get_config(...)`
3.  Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4.  Build `trainer = Trainer(config)`

In [3]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'model.batch_size=1',
        'model.block_size=128',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=bfloat16',
        'train.max_iters=500',
        'train.log_interval=10',
        'train.eval_interval=50',
    ]
)
config = instantiate(cfg)

[ 2025-07-22 22:23:53,523398 ][ I ][ wordplay / configs : 81 ] Setting HF_DATASETS_CACHE to /Users/samforeman/projects/saforem2/wordplay/.cache/huggingface/ datasets

[ 2025-07-22 22:23:53,530105 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-22 22:23:53,531006 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-22 22:23:53,531724 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-22 22:23:53,532329 ][ I ][ ezpz / dist : 1159 ] Using fw = 'ddp' with torch_ { device,backend } = { cpu, gloo }

...

[ 2025-07-22 22:23:53,639060 ][ I ][ wordplay / configs : 317 ] Loading val from /Users/samforeman/projects/saforem2/wordplay/data/shakespeare_char/ val.bin

[ 2025-07-22 22:23:53,645259 ][ I ][ wordplay / configs : 442 ] Tokens per iteration: 128

[ 2025-07-22 22:23:53,645936 ][ W ][ ezpz / dist : 639 ] Caught TORCH_DEVICE = cpu from environment!

[ 2025-07-22 22:23:53,646471 ][ I ][ wordplay / configs : 465 ] Using self. ptdtype = torch .bfloat16 on self. device_type = 'cpu'

[ 2025-07-22 22:23:53,647031 ][ I ][ wordplay / configs : 471 ] Initializing a new model from scratch

### Build `Trainer` object

In [4]:
trainer = Trainer(config)

[ 2025-07-22 22:23:53,682989 ][ I ][ wordplay / trainer : 248 ] Initializing a new model from scratch

[ 2025-07-22 22:23:53,859416 ][ I ][ wordplay / model : 255 ] number of parameters: 10. 65M

[ 2025-07-22 22:23:53,860725 ][ I ][ wordplay / trainer : 266 ] Model size: num_params = 10646784

[ 2025-07-22 22:23:53,861429 ][ I ][ wordplay / model : 445 ] num decayed parameter tensors: 26 , with 10 , 690 , 944 parameters

[ 2025-07-22 22:23:53,862054 ][ I ][ wordplay / model : 449 ] num non-decayed parameter tensors: 13 , with 4 , 992 parameters

...

[ 2025-07-22 22:23:53,863498 ][ C ][ wordplay / trainer : 318 ] "devid='cpu:0'"

[ 2025-07-22 22:23:53,870522 ][ I ][ wordplay / trainer : 358 ] • self. model = GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

[ 2025-07-22 22:23:53,873563 ][ I ][ wordplay / trainer : 359 ] • self. grad_scaler = None

[ 2025-07-22 22:23:53,874438 ][ I ][ wordplay / trainer : 360 ] • self. model_engine = GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

[ 2025-07-22 22:23:53,877064 ][ I ][ wordplay / trainer : 361 ] • self. optimizer = AdamW ( 
Parameter Group 0 
 amsgrad: False 
 betas: ( 0.9 , 0.99 ) 
 capturable: False 
 decoupled_weight_decay: True 
 differentiable: False 
 eps: 1e-08 
 foreach: None 
 fused: None 
 lr: 0.001 
 maximize: False 
 weight_decay: 0.1 

Parameter Group 1 
 amsgrad: False 
 betas: ( 0.9 , 0.99 ) 
 capturable: False 
 decoupled_weight_decay: True 
 differentiable: False 
 eps: 1e-08 
 foreach: None 
 fused: None 
 lr: 0.001 
 maximize: False 
 weight_decay: 0.0 
 )

## Prompt (**prior** to training)

In [5]:
query = "What is an LLM?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-22 22:23:55,957318 ][ I ][ ipykernel_52985 / 3496000222 : 9 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-22 22:23:55,958320 ][ I ][ ipykernel_52985 / 3496000222 : 10 : ezpz.log ] [ 'response' ] :

What is an LLM?A,,osy'exx.ff.fpppxv;;'vt3QjYhhvvYAhowQwwQ,eqeqG;X.YqqQSZQWLsyccccj:ZhaooxkkcfkZ
ffop- f,hqWl
oocpppUqAQ;cc' 'bQqcWAttrqerrwyqqsrqttqYeqWQs' tottcqestbqbbrpWbWYAppppBqfhcqqYqqM?qttqQU'gYe?A.. 'S' rtppW'fJf;??qn.pwrrrqqfA;!!A,,,AtqqqqbW;bSoW;;?;;;qQ;;cIA. 'M;' 'g

## Train Model

|  name  |        description         |
|:------:|:--------------------------:|
| `step` |   Current training step    |
| `loss` |         Loss value         |
|  `dt`  | Time per step (in **ms**)  |
| `sps`  |     Samples per second     |
| `mtps` |  (million) Tokens per sec  |
| `mfu`  | Model Flops utilization[1] |

^legend: #tbl-legend

[1] in units of A100 `bfloat16` peak FLOPS

In [6]:
trainer.config.device_type

'cpu'

In [7]:
from rich import print

print(trainer.model)

GPT ( 
 ( transformer ) : ModuleDict ( 
 ( wte ) : Embedding ( 65 , 384 ) 
 ( wpe ) : Embedding ( 128 , 384 ) 
 ( drop ) : Dropout ( p = 0.2 , inplace = False ) 
 ( h ) : ModuleList ( 
 ( 0 - 5 ) : 6 x Block ( 
 ( ln_1 ) : LayerNorm () 
 ( attn ) : CausalSelfAttention ( 
 ( c_attn ) : Linear ( in_features = 384 , out_features = 1152 , bias = False ) 
 ( c_proj ) : Linear ( in_features = 384 , out_features = 384 , bias = False ) 
 ( attn_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ( resid_dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ( ln_2 ) : LayerNorm () 
 ( mlp ) : MLP ( 
 ( c_fc ) : Linear ( in_features = 384 , out_features = 1536 , bias = False ) 
 ( act_fn ) : GELU ( approximate = 'none' ) 
 ( c_proj ) : Linear ( in_features = 1536 , out_features = 384 , bias = False ) 
 ( dropout ) : Dropout ( p = 0.2 , inplace = False ) 
 ) 
 ) 
 ) 
 ( ln_f ) : LayerNorm () 
 ) 
 ( lm_head ) : Linear ( in_features = 384 , out_features = 65 , bias = False ) 
 )

## (partial) Training:

We’ll first train for 500 iterations and then evaluate the models
performance on the same prompt:

> What is an LLM?

In [8]:
trainer.train(train_iters=500)

Training Legend 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ abbr ┃ desc ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ step │ Current training iteration │
│ loss │ Loss value │
│ dt │ Elapsed time per training step │
│ dtf │ Elapsed time per forward step │
│ dtb │ Elapsed time per backward step │
│ sps │ Samples per second │
│ sps_per_gpu │ Samples per second (per GPU) │
│ tps │ Tokens per second │
│ tps_per_gpu │ Tokens per second (per GPU) │
│ mfu │ Model flops utilization │
└─────────────┴────────────────────────────────┘

[ 2025-07-22 22:23:58,164358 ][ I ][ wordplay / trainer : 834 ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-22 22:23:58,165261 ][ I ][ wordplay / trainer : 838 ] [ 'response' ] :

What is an LLM?wCw'.AAAfxo.. 'yfAQfppyybvFYerr.MfYZAcLyQQCkkexx-3lllrpMqxkko-rZx3b' 3j -ffSSoqq3hhdf'Q' 'aq' wqqsoKZb'ec3ZAAA;;o,qff.. 'fArttgbYtturcbcSYrS-Fff' wwwerwPgJ;.e;yY-SpuyeexqYqgQtpMSYqYgbtQqq' '' ;pfsw, ';oA;qqeqcckSAo,,rooMgyQha''' fAA..gg;;'ggtSvrupptkeweqqcqqkk-SvYYIv

[ 2025-07-22 22:24:01,981608 ][ I ][ wordplay / trainer : 901 ] step = 10 loss = 4.28757 dt = 0.0178168 dtf = 0.0175137 dtb = 0.000120083 sps = 56.127 sps_per_gpu = 56.127 tps = 7184.25 tps_per_gpu = 7184.25 mfu = 0.155243

[ 2025-07-22 22:24:02,159338 ][ I ][ wordplay / trainer : 901 ] step = 20 loss = 4.28569 dt = 0.0180799 dtf = 0.0177861 dtb = 0.000114416 sps = 55.31 sps_per_gpu = 55.31 tps = 7079.68 tps_per_gpu = 7079.68 mfu = 0.155018

...

[ 2025-07-22 22:24:53,331371 ][ I ][ wordplay / trainer : 901 ] step = 460 loss = 4.24269 dt = 0.0171837 dtf = 0.0169337 dtb = 9.94999e-05 sps = 58.1945 sps_per_gpu = 58.1945 tps = 7448.9 tps_per_gpu = 7448.9 mfu = 0.143626

[ 2025-07-22 22:24:53,514814 ][ I ][ wordplay / trainer : 901 ] step = 470 loss = 4.26877 dt = 0.0177474 dtf = 0.0174593 dtb = 0.000126833 sps = 56.3462 sps_per_gpu = 56.3462 tps = 7212.32 tps_per_gpu = 7212.32 mfu = 0.144848

[ 2025-07-22 22:24:53,703659 ][ I ][ wordplay / trainer : 901 ] step = 480 loss = 4.19188 dt = 0.0203032 dtf = 0.0199089 dtb = 0.00012725 sps = 49.2533 sps_per_gpu = 49.2533 tps = 6304.42 tps_per_gpu = 6304.42 mfu = 0.143987

[ 2025-07-22 22:24:54,084714 ][ I ][ wordplay / trainer : 901 ] step = 490 loss = 4.22611 dt = 0.023646 dtf = 0.0232924 dtb = 0.000135083 sps = 42.2905 sps_per_gpu = 42.2905 tps = 5413.18 tps_per_gpu = 5413.18 mfu = 0.141285

[ 2025-07-22 22:24:54,332990 ][ I ][ wordplay / trainer : 901 ] step = 500 loss = 4.21804 dt = 0.0283218 dtf = 0.0270621 dtb = 0.00013475 sps = 35.3084 sps_per_gpu = 35.3084 tps = 4519.48 tps_per_gpu = 4519.48 mfu = 0.136923

In [9]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-22 22:24:57,059118 ][ I ][ ipykernel_52985 / 1425179755 : 12 : ezpz.log ] took: 1. 9380s

[ 2025-07-22 22:24:57,062558 ][ I ][ ipykernel_52985 / 1425179755 : 13 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-22 22:24:57,063296 ][ I ][ ipykernel_52985 / 1425179755 : 14 : ezpz.log ] [ 'response' ] :

What is an LLM?fwll

b3afqbZZI,r oppq3A33QoUUye-fwC'3b3. ',A' .hhPlVXXqeQyCCC;xfssc;wTTTTcdGoeehQOCXXXB'KZ--qehoF3AqfqqW
cQAcceffGG, 'fSJpppww,txMgQs;;;?qf' fSSrpcg?s,A'rr,aso? '' o'MtQrrSSgqfttggSc' 'Wb' qA,.Apcbb???;pYYySQ'agggScWQgbqWfqYroffSYSYhqfk' 'qfAA,sgWlnZ:pt,JynS' gJZes

## Resume Training…

In [10]:
trainer.train()

[ 2025-07-22 22:24:59,754437 ][ I ][ wordplay / trainer : 834 ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-22 22:24:59,755986 ][ I ][ wordplay / trainer : 838 ] [ 'response' ] :

What is an LLM?wZbbbT'3weew, 'foBB.qWWlpwes.qqQevFAA.bbvFF-AkacWWfYhx3fooB''' ';vveesppWW
eeWA3ZZppPZe;dCCvres ;ecc--Ws'cqor,JZVVVCCeepfqqWxApBBBBhh;;JeQhMMss,,wshrhW?BiMWYqqwwwAASSwrrroo,rqtWseMq.Ak'ofA,, 't,,..hh;xx' ?sAq';cqxrqWkeMqt'gzAAxhrpqt'g't;?btoseq-pqq'qAtttt,eqrM

[ 2025-07-22 22:25:02,884658 ][ I ][ wordplay / trainer : 769 ] Saving checkpoint to: /Users/samforeman/projects/saforem2/intro-hpc-bootcamp-2025/content/02-llms/ 7-shakespeare-example

[ 2025-07-22 22:25:02,891985 ][ I ][ wordplay / trainer : 770 ] Saving model to: /Users/samforeman/projects/saforem2/intro-hpc-bootcamp-2025/content/02-llms/7-shakespeare-example/ model.pth

[ 2025-07-22 22:25:03,057365 ][ I ][ wordplay / configs : 141 ] Appending /Users/samforeman/projects/saforem2/intro-hpc-bootcamp-2025/content/02-llms/ 7-shakespeare-example to /Users/samforeman/projects/saforem2/wordplay/src/ckpts/ checkpoints.log

...

[ 2025-07-22 22:26:03,626057 ][ I ][ wordplay / trainer : 901 ] step = 960 loss = 4.225 dt = 0.019989 dtf = 0.0196397 dtb = 0.0001325 sps = 50.0276 sps_per_gpu = 50.0276 tps = 6403.54 tps_per_gpu = 6403.54 mfu = 0.133264

[ 2025-07-22 22:26:03,832092 ][ I ][ wordplay / trainer : 901 ] step = 970 loss = 4.17741 dt = 0.0182224 dtf = 0.0179159 dtb = 0.000119958 sps = 54.8776 sps_per_gpu = 54.8776 tps = 7024.33 tps_per_gpu = 7024.33 mfu = 0.135116

[ 2025-07-22 22:26:04,030898 ][ I ][ wordplay / trainer : 901 ] step = 980 loss = 4.1707 dt = 0.0182135 dtf = 0.0179185 dtb = 0.000116791 sps = 54.9042 sps_per_gpu = 54.9042 tps = 7027.74 tps_per_gpu = 7027.74 mfu = 0.136791

[ 2025-07-22 22:26:04,235641 ][ I ][ wordplay / trainer : 901 ] step = 990 loss = 4.1891 dt = 0.0191646 dtf = 0.0187604 dtb = 0.000157208 sps = 52.1795 sps_per_gpu = 52.1795 tps = 6678.97 tps_per_gpu = 6678.97 mfu = 0.137544

[ 2025-07-22 22:26:04,441023 ][ I ][ wordplay / trainer : 901 ] step = 1000 loss = 4.2423 dt = 0.02126 dtf = 0.0196627 dtb = 0.000133416 sps = 47.0366 sps_per_gpu = 47.0366 tps = 6020.68 tps_per_gpu = 6020.68 mfu = 0.1368

## Evaluate Model

In [11]:
import time

query = "What is an LLM?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=2,
    display=False
)
logger.info(f'took: {time.perf_counter() - t0:.4f}s')
logger.info(f"['prompt']: '{query}'")
logger.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[ 2025-07-22 22:26:07,057847 ][ I ][ ipykernel_52985 / 582817405 : 12 : ezpz.log ] took: 2. 0044s

[ 2025-07-22 22:26:07,061726 ][ I ][ ipykernel_52985 / 582817405 : 13 : ezpz.log ] [ 'prompt' ] : 'What is an LLM?'

[ 2025-07-22 22:26:07,062941 ][ I ][ ipykernel_52985 / 582817405 : 14 : ezpz.log ] [ 'response' ] :

What is an LLM?ZxxA--- 'aaaaeeewAAAAA''' qqqqqqqqqqqqaeeqqqqqq' '333qqAAA33akkk''qqqqqorrrrrrrrrrqqqqqqq.qe333aaaqqqqqf..qqqqqqq3333333-qqqqbbb''ggSSpMMMqqqqMMqqqqqqqqWW;?;?;?;???;;??MMMM;;;;;;??;;;;;;;;''''';??qqqqqqqW;;''''''''''''' ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'tttttMM